In [ ]:
def plotly_template(template_specs):

    import plotly.graph_objects as go

    # Layout
    fig_template=go.layout.Template()
    fig_template.layout = (go.Layout(# general
                                    font_family=template_specs['font'],
                                    font_size=template_specs['axes_font_size'],
                                    plot_bgcolor=template_specs['bg_col'],

                                    # x axis
                                    xaxis_visible=True,
                                    xaxis_linewidth=template_specs['axes_width'],
                                    xaxis_color= template_specs['axes_color'],
                                    xaxis_showgrid=False,
                                    xaxis_ticks="outside",
                                    xaxis_ticklen=0,
                                    xaxis_tickwidth = template_specs['axes_width'],
                                    xaxis_title_font_family=template_specs['font'],
                                    xaxis_title_font_size=template_specs['title_font_size'],
                                    xaxis_tickfont_family=template_specs['font'],
                                    xaxis_tickfont_size=template_specs['axes_font_size'],
                                    xaxis_zeroline=False,
                                    xaxis_zerolinecolor=template_specs['axes_color'],
                                    xaxis_zerolinewidth=template_specs['axes_width'],
                                    xaxis_range=[0,1],
                                    xaxis_hoverformat = '.1f',
                                    
                                    # y axis
                                    yaxis_visible=False,
                                    yaxis_linewidth=0,
                                    yaxis_color= template_specs['axes_color'],
                                    yaxis_showgrid=False,
                                    yaxis_ticks="outside",
                                    yaxis_ticklen=0,
                                    yaxis_tickwidth = template_specs['axes_width'],
                                    yaxis_tickfont_family=template_specs['font'],
                                    yaxis_tickfont_size=template_specs['axes_font_size'],
                                    yaxis_title_font_family=template_specs['font'],
                                    yaxis_title_font_size=template_specs['title_font_size'],
                                    yaxis_zeroline=False,
                                    yaxis_zerolinecolor=template_specs['axes_color'],
                                    yaxis_zerolinewidth=template_specs['axes_width'],
                                    yaxis_hoverformat = '.1f',
                                    ))

    # Annotations
    fig_template.layout.annotationdefaults = go.layout.Annotation(
                                    font_color=template_specs['axes_color'],
                                    font_family=template_specs['font'],
                                    font_size=template_specs['title_font_size'])

    return fig_template
    

In [ ]:
def compute_pmf(ecc, angle, size):

    # general import
    import numpy as np
    import scipy.io

    # model imports
    # -------------
    from prfpy.stimulus import PRFStimulus2D
    from prfpy.model import Iso2DGaussianModel
    from prfpy.fit import Iso2DGaussianFitter

    ppd = 52                                                        # nb of pixels per degrees
    eyemove_height_dva = 30                                         # eye movement are height in dva
    screen_width_px,screen_height_px = 1080,1080                    # screen size
    ratio_out = eyemove_height_dva/(screen_height_px/ppd)           # ratio to compute of out of screen pixel size eye movement can be made
    screen_height_cm = 43.5 * ratio_out
    screen_width_cm = 43.5 * ratio_out
    screen_distance_cm = 120
    TR = 1.2
    TRs = 208
    sub_task = 'sac'
    seq_num = 1
    vd_file = scipy.io.loadmat("../data/vd/pMF{}_vd_seq{}.mat".format(sub_task, seq_num))

    # determine model
    vd = vd_file['stim'].transpose([1,0,2])
    stimulus = PRFStimulus2D(   screen_size_cm=screen_width_cm,
                                screen_distance_cm=screen_distance_cm,
                                design_matrix=vd,
                                TR=TR)
    gauss_model = Iso2DGaussianModel(stimulus=stimulus, normalize_RFs = True)

    # model parameters
    angle_rad = np.deg2rad(angle)
    mu_x = np.cos(angle_rad)*ecc
    mu_y = np.sin(angle_rad)*ecc
    beta = 1
    baseline = 0

    # define model time course
    tc_row = gauss_model.return_prediction( mu_x=np.array(mu_x), 
                                            mu_y=np.array(mu_y),
                                            size=np.array(size), 
                                            beta=np.array(1), 
                                            baseline=np.array(baseline))
    tc_norm = ((tc_row - np.min(tc_row)) / (np.max(tc_row) - np.min(tc_row))) - ((baseline - np.min(tc_row)) / (np.max(tc_row) - np.min(tc_row))) 

    # define model motor field
    gauss_fitter = Iso2DGaussianFitter(data=tc_norm, model=gauss_model)
    gauss_fitter.grid_fit(ecc_grid=ecc, polar_grid = angle_rad, size_grid=size)
    gauss_model.create_rfs()
    pmf = gauss_model.grid_rfs[0,:,:]
    pmf_norm = ((pmf - np.min(pmf)) / (np.max(pmf) - np.min(pmf)))*255

    return (tc_norm, pmf_norm)


In [ ]:
def plot(ecc=2.5, angle=90, size=1):
    tc_norm, pmf_norm = compute_pmf(ecc, angle, size)
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np

    # general figure settings
    template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",          # figure axes color
                            axes_width=2,                           # figureaxes line width
                            axes_font_size=15,                      # font size of axes
                            bg_col="rgba(255, 255, 255, 1)",        # figure background color
                            font='Helvetica',                       # general font used
                            title_font_size=18,                     # font size of titles
                            plot_width=1.5,                         # plot line width
                            )
    fig_template = plotly_template(template_specs)

    fig_height, fig_width = 250,1000
    rows, cols = 1, 4

    x_label_tc, y_label_tc = '# TR', 'Signal (a.u.)'
    x_range_tc, y_range_tc = [0,208], [-0.5,1]
    x_tickvals_tc = np.linspace(x_range_tc[0],x_range_tc[1],14)
    y_tickvals_tc = np.linspace(y_range_tc[0],y_range_tc[1],4)
    x_ticktexts_tc = ['{:g}'.format(x) for x in x_tickvals_tc]
    y_ticktexts_tc = ['{:g}'.format(x) for x in y_tickvals_tc]

    ppd = 52
    amp_eyemov = np.linspace(2.5,10,4)*ppd                          # smooth pursuit and saccade amplitude
    pix_ratio = 0.125                                               # ratio of pixel motor design for later fit

    # axis settings
    screen_model_width, screen_model_height = pmf_norm.shape[0], pmf_norm.shape[1]
    x_label_map, y_label_map = 'Hor. coord. (dva)', 'Ver. coord. (dva)',
    x_range_map, y_range_map = [0,screen_model_width], [0,screen_model_height]
    x_tickvals_map, y_tickvals_map = np.linspace(0,screen_model_width-1,5), np.linspace(0,screen_model_height-1,5)
    x_ticktexts_map = ['{:g}'.format(x) for x in np.linspace(-15,15,5)]
    y_ticktexts_map = ['{:g}'.format(x) for x in np.linspace(15,-15,5)]

    # subplot settings
    row_heights = [1]
    column_widths = [1,1,1,1]

    sb_specs = [[{'colspan':3},None,None,{}]]
    fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.1, horizontal_spacing=0.5/5,
                        column_widths=column_widths, row_heights=row_heights, shared_yaxes=False)

    # draw model timecourse
    fig.add_trace(go.Scatter(y=tc_norm[0],
                                mode='lines', name='model',showlegend=False,
                                line_color='rgba(68,1,84,1)'),row=1, col=1)

    # draw model map
    fig.add_trace(go.Heatmap(   z=pmf_norm,
                                colorscale='viridis',
                                showscale=False),
                                row=1, 
                                col=4)

    for rad_num in [0,1,2,3]:
        rad_circle = amp_eyemov[rad_num]*pix_ratio
        fig.add_shape(  type="circle",
                        x0=screen_model_width/2-rad_circle, y0=screen_model_height/2-rad_circle,
                        x1=screen_model_width/2+rad_circle, y1=screen_model_height/2+rad_circle,
                        line_color='white',
                        line_dash='dot',
                        line_width=1,
                        row = 1,
                        col = 4)

    # figure layout
    fig.layout.update(  # figure settings
                        template=fig_template, width=fig_width, height=fig_height, margin_l=70, margin_r=20, margin_t=20, margin_b=70,
                        
                        # timecourse
                        yaxis_visible=True, yaxis1_linewidth=template_specs['axes_width'], yaxis1_title_text=y_label_tc, 
                        yaxis1_range=y_range_tc, yaxis1_ticklen=8, yaxis1_tickvals=y_tickvals_tc, yaxis1_ticktext=y_ticktexts_tc,
                        xaxis1_visible=True, xaxis1_linewidth=template_specs['axes_width'], xaxis1_title_text=x_label_tc, 
                        xaxis1_range=x_range_tc, xaxis1_ticklen=8, xaxis1_tickvals=x_tickvals_tc, xaxis1_ticktext=x_ticktexts_tc,

                        xaxis2_visible=True, xaxis2_linewidth=template_specs['axes_width'], xaxis2_title_text=x_label_map, 
                        xaxis2_range=x_range_map, xaxis2_ticklen=8, xaxis2_ticktext=x_ticktexts_map, xaxis2_tickvals=x_tickvals_map,
                        yaxis2_visible=True, yaxis2_linewidth=template_specs['axes_width'], yaxis2_title_text=y_label_map, 
                        yaxis2_range=y_range_map, yaxis2_ticklen=8, yaxis2_ticktext=y_ticktexts_map, yaxis2_tickvals=y_tickvals_map,
                        yaxis2_autorange='reversed',
                        
                        )

    fig.show()

In [ ]:
# 1
plot(ecc=2.5, angle=180, size=0.5)

In [ ]:
# 2
plot(ecc=2.5, angle=270, size=0.5)

In [ ]:
# 3
plot(ecc=2.5, angle=0, size=0.5)

In [ ]:
# 4
plot(ecc=2.5, angle=90, size=0.5)

In [ ]:
# plot prf visual design
import scipy.io
import plotly.express as px
vd_file = scipy.io.loadmat("../data/vd/pMF{}_vd_seq{}.mat".format('sac', 1))
visual_dm = vd_file['stim'].transpose([2,1,0])
fig = px.imshow(visual_dm[19,:,:], color_continuous_scale='gray',zmin=0,zmax=255)
fig.show()

In [154]:
def compute_screen(screen, sigma, mu, rot):

    import numpy as np

    x, y = np.meshgrid(np.arange(0,screen[0],1), np.arange(0,screen[1],1))
    x,y = x - mu[0], y - mu[1]
    rot_mat = np.array([[np.cos(rot), np.sin(rot)],
                        [-np.sin(rot), np.cos(rot)]])
    mult = np.dot(rot_mat, np.array([x.ravel(),y.ravel()]))
    x = mult[0,:].reshape(x.shape)
    y = mult[1,:].reshape(y.shape)
    z = (1/(2*np.pi*sigma[0]*sigma[1]) * np.exp(-((x)**2/(2*sigma[0]**2) + (y)**2/(2*sigma[1]**2))))
    z_norm = (z - np.min(z)) / (np.max(z) - np.min(z))
    z_col = np.round(z_norm*255)
    
    return z_col


import numpy as np
from PIL import Image 
import itertools
import scipy.io
sp_radial_sd = [0.5, 0.5, 0.5, 0.5]
sp_tangential_sd = [0.5, 0.5, 0.5, 0.5]

sac_radial_sd = [0.5, 0.5, 0.5, 0.5]
sac_tangential_sd = [0.5, 0.5, 0.5, 0.5]
sub_task ='sac'

# compute parameters
trs_break = 16                                                  # trs during break period
trs_eye_mov = 32                                                # trs during eye movement period
ppd = 52                                                        # nb of pixels per degrees
TRs = 208                                                       # total number of TR
screen_width_px, screen_height_px = 1080,1080                   # screen width and height in pixel
eyemove_height_dva = 30                                         # eye movement are height in dva
pix_ratio = 0.125                                               # ratio of pixel motor design for later fit
ratio_out = eyemove_height_dva/(screen_height_px/ppd)           # ratio to compute of out of screen pixel size eye movement can be made
screen = [screen_width_px,screen_height_px]                     # screen resolution in pixels (use height of pRF)
screen = [int(screen[0]*ratio_out),int(screen[1]*ratio_out)]    # screen recomputed size
center = [screen[0]/2,screen[1]/2]                              # screen center 
dir_sp = np.deg2rad(np.arange(0,360,22.5))                      # smooth pursuit directions in radian
dir_sac = np.deg2rad(np.arange(0,360,22.5)+180)                 # saccade directions in radian
sp_amp = np.linspace(2.5,10,4)*ppd                              # smooth pursuit amplitude/speed
sac_amp = np.linspace(2.5,10,4)*ppd                             # saccade amplitude

sigma_sp  =[[sp_radial_sd[0]*ppd, sp_tangential_sd[0]*ppd],     # smooth pursuit area sd for first amplitude
            [sp_radial_sd[1]*ppd, sp_tangential_sd[1]*ppd],     # smooth pursuit area sd for second amplitude
            [sp_radial_sd[2]*ppd, sp_tangential_sd[2]*ppd],     # smooth pursuit area sd for third amplitude
            [sp_radial_sd[3]*ppd, sp_tangential_sd[3]*ppd]]     # smooth pursuit area sd for fourth amplitude

sigma_sac =[[sac_radial_sd[0]*ppd, sac_tangential_sd[0]*ppd],   # saccade area sd for first amplitude
            [sac_radial_sd[1]*ppd, sac_tangential_sd[1]*ppd],   # saccade area sd for second amplitude
            [sac_radial_sd[2]*ppd, sac_tangential_sd[2]*ppd],   # saccade area sd for third amplitude
            [sac_radial_sd[3]*ppd, sac_tangential_sd[3]*ppd]]   # saccade area sd for fourth amplitude

seq_order = [5,1,5,2,5,3,5,4,5]
stim = np.zeros((int(screen[1]),int(screen[0]),TRs))
tr_num = 0
eyemov_num = 0

for seq in seq_order:
    if seq == 5:
        for tr in np.arange(0,trs_break,1):
            # make blank image
            stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
            tr_num += 1
    else:
        num_sp, num_sac = 0, 0
        for tr in np.arange(0,trs_eye_mov,1):

            if np.mod(tr,2)==False:

                if sub_task == 'sp':
                    # get sp center coordinate in retinal coordinates
                    mu = [center[0] + np.cos(dir_sp[num_sp])*sp_amp[seq-1], center[1] - np.sin(dir_sp[num_sp])*sp_amp[seq-1]]
                    rot = -dir_sp[num_sp]
                    sigma = sigma_sp[seq-1]
                    stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)
                    num_sp += 1
                else:
                    stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
                tr_num += 1
                
            else:
                if sub_task == 'sac': 
                    # get saccade center coordinate in retinal coordinates
                    mu = [center[0] + np.cos(dir_sac[num_sac])*sac_amp[seq-1], center[1] - np.sin(dir_sac[num_sac])*sac_amp[seq-1]]
                    rot = -dir_sac[num_sac]
                    sigma = sigma_sac[seq-1]
                    stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)
                    num_sac += 1
                else:
                    stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
                tr_num += 1


# resize it for fit
screen_resize = [int(screen[0]*pix_ratio),int(screen[1]*pix_ratio)]
center_resize = [screen_resize[0]/2,screen_resize[1]/2]
stim_resize = np.zeros([screen_resize[0],screen_resize[1],TRs])
for tr in np.arange(0,TRs,1):
    stim_resize[:,:,tr] = np.array(Image.fromarray(stim[:,:,tr]).convert('RGB').resize(size=screen_resize))[:,:,0]



In [155]:
np.arange(0,360,22.5)+180

array([180. , 202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5, 360. ,
       382.5, 405. , 427.5, 450. , 472.5, 495. , 517.5])